In [1]:
#I made a module called spreader that takes the non numeric values and makes columns out of them and fills them with 1 
include("Spreader.jl")

Main.Spreader

In [25]:
using CSV
using DataFrames
using .Spreader
using Statistics

In [3]:
df = DataFrame(CSV.File("Adult.csv", missingstring="?"))
println("Number of rows: $(string(nrow(df)))")

Number of rows: 48842


In [4]:
WORKERS = 123_100_000
print("Total Workers: $(WORKERS)")

Total Workers: 123100000

In [5]:
total_weight = sum(df[:,:fnlwgt])
print("Total weight: $(total_weight)" )

Total weight: 9263575662

In [6]:
weight_factor = WORKERS/total_weight
print("Weight factor: $(weight_factor)")

Weight factor: 0.013288605231019702

In [7]:
# This is where I use my magical spread function
spread!(df,"workclass")
spread!(df,"education")
spread!(df,"marital_status")
spread!(df,"occupation")
spread!(df,"race")
spread!(df,"sex")
spread!(df,"native_country")
spread!(df,"class")
println("Number of columns after spread operations: " * string(ncol(df)))

Number of columns after spread operations: 109


In [8]:
# The following lines multiply the new columns by fnlwgh and the calculated weight factor
df[!,Between(:Private,end)] = df[!,Between(:Private,end)] .* (df[!,:fnlwgt] * weight_factor)
df[!,Between(:Private,end)] = Int64.(round.(df[!,Between(:Private,end)]));

#This makes a total column that is actual total people
transform!(df, :fnlwgt => ((x) -> return 0) => :Total)
df[!,:Total] = df[!,:Total] = (df[!,:fnlwgt] * weight_factor)
df[!,:Total] = Int64.(round.(df[!,:Total]));

In [21]:
# Describe DataFrame
# Smmarry stats for all columns
describe(df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,age,38.6436,17,37.0,90,0,Int64
2,workclass,,Federal-gov,,Without-pay,2799,"Union{Missing, String31}"
3,fnlwgt,1.89664e5,12285,1.78144e5,1490400,0,Int64
4,education,,10th,,Some-college,0,String15
5,education_num,10.0781,1,10.0,16,0,Int64
6,marital_status,,Divorced,,Widowed,0,String31
7,occupation,,Adm-clerical,,Transport-moving,2809,"Union{Missing, String31}"
8,relationship,,Husband,,Wife,0,String15
9,race,,Amer-Indian-Eskimo,,White,0,String31


In [22]:
# Specific State for all columns
describe(df,:mean,:std)

Row,variable,mean,std
,Symbol,Union…,Union…
1,age,38.6436,13.7105
2,workclass,,
3,fnlwgt,1.89664e5,105604.0
4,education,,
5,education_num,10.0781,2.57097
6,marital_status,,
7,occupation,,
8,relationship,,
9,race,,


In [26]:
# Apply stats function to all columns (Variance)
describe(df, var => :var)

Row,variable,var
,Symbol,Union…
1,age,187.978
2,workclass,
3,fnlwgt,1.11522e10
4,education,
5,education_num,6.6099
6,marital_status,
7,occupation,
8,relationship,
9,race,


In [27]:
# First five rows
first(df,5)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,18,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯


In [28]:
# Last five rows
last(df,5)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,3419,0,0,0,0,0,0,0,0,3419,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3419,0,0,0,0,0,0,0,0,0,0,0,0,0,3419,0,0,0,0,0,0,3419,0,0,0,0,3419,3419,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,2051,0,0,0,0,0,0,0,2051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2051,0,0,0,0,0,2051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2051,0,0,0,2051,0,2051,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,2019,0,0,0,0,0,0,0,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,0,0,0,0,0,0,0,0,0,0,2019,0,0,0,0,0,0,0,0,2019,0,0,0,0,2019,2019,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,2678,0,0,0,0,0,0,0,2678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2678,0,0,0,0,0,0,0,0,0,0,0,0,2678,0,0,0,0,0,0,0,0,2678,0,0,0,2678,0,2678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K,0,0,0,0,0,3826,0,0,3826,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3826,0,0,0,0,0,0,0,0,0,0,0,0,3826,0,0,0,0,0,0,0,3826,0,0,0,0,3826,3826,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯


In [29]:
# Unique rows
unique(df)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,18,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0

In [43]:
# Return data frame with unique rows
unique(df,[:age, :occupation])

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,18,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0

In [30]:
# Filter for sex = Male
filter(:sex => ==("Male"), df)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0

In [31]:
# Different filtering syntax
filter(row -> row.sex == "Male", df)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0

In [ ]:
# Subsetting?
#Only works with booleans
#subset(df, :Private)

In [35]:
# More subsetting?
subset(df, :sex => x -> x .== "Male")

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0

In [36]:
# Indexing
df[6:10,1:5]

Row,age,workclass,fnlwgt,education,education_num
,Int64,String31?,Int64,String15,Int64
1,34,Private,198693,10th,6
2,29,missing,227026,HS-grad,9
3,63,Self-emp-not-inc,104626,Prof-school,15
4,24,Private,369667,Some-college,10
5,55,Private,104996,7th-8th,4


In [39]:
# Indexing syntax all sex = Male
df[df.sex .== "Male", :]

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0

In [41]:
# Find first row where age = 30
df[findfirst(==(30), df.age), :]

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
61,30,Private,101135,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,50,United-States,<=50K,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,1344,0,0,0,0,1344,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯


In [42]:
# Find all where age = 30
df[findall(==(30),df.age),:]

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,30,Private,101135,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,50,United-States,<=50K,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,1344,0,0,0,0,0,0,0,1344,0,0,0,0,1344,1344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,30,Private,229636,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,Mexico,<=50K,3052,0,0,0,0,0,0,0,3052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3052,0,0,0,0,0,3052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3052,0,0,0,3052,0,0,0,0,3052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,30,Private,142921,Assoc-acdm,12,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,40,United-States,<=50K,1899,0,0,0,0,0,0,0,0,1899,0,0,0,0,0,0,0,0,0,0,0,0,0,1899,0,0,0,0,0,0,0,0,0,0,1899,0,0,0,0,0,0,0,0,0,0,1899,0,0,0,0,1899,1899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,30,State-gov,260782,HS-grad,9,Never-married,Other-service,Not-in-family,White,Male,0,0,40,United-States,<=50K,0,0,0,0,3465,0,0,0,3465,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3465,0,0,0,0,0,0,0,0,0,3465,0,0,0,0,0,0,0,0,0,0,0,3465,0,0,0,3465,0,3465,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,30,Private,296462,HS-grad,9,Never-married,Exec-managerial,Not-in-family,Black,Male,0,0,40,United-States,<=50K,3940,0,0,0,0,0,0,0,3940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3940,0,0,0,0,0,0,0,0,0,0,0,0,0,3940,0,0,0,0,0,0,3940,0,0,0,0,3940,0,3940,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,30,Private,236543,9th,5,Married-civ-spouse,Other-service,Husband,White,Male,0,0,32,El-Salvador,>50K,3143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3143,0,0,0,0,0,3143,0,0,0,0,0,0,0,0,3143,0,0,0,0,0,0,0,0,0,0,0,3143,0,0,0,3143,0,0,0,0,0,0,0,0,0,0,0,3143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,30,Private,238186,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Other-relative,White,Male,0,2057,48,United-States,<=50K,3165,0,0,0,0,0,0,0,3165,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3165,0,0,0,0,0,3165,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3165,0,0,0,3165,0,3165,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,30,Private,169269,11th,7,Never-married,Handlers-cleaners,Other-relative,White,Male,0,1721,38,Puerto-Rico,<=50K,2249,0,0,0,0,0,0,2249,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2249,0,0,0,0,0,0,0,0,0,0,0,0

In [44]:
# select sex column
select(df, :sex)

Row,sex
,String7
1,Male
2,Male
3,Male
4,Male
5,Female
6,Male
7,Male
8,Male
9,Female


In [45]:
# select sex column
select(df,"sex")

Row,sex
,String7
1,Male
2,Male
3,Male
4,Male
5,Female
6,Male
7,Male
8,Male
9,Female


In [46]:
# select sex and age column
select(df,[:sex,:age])

Row,sex,age
,String7,Int64
1,Male,25
2,Male,38
3,Male,28
4,Male,44
5,Female,18
6,Male,34
7,Male,29
8,Male,63
9,Female,24


In [47]:
# select columns 2-5
select(df,2:5)

Row,workclass,fnlwgt,education,education_num
,String31?,Int64,String15,Int64
1,Private,226802,11th,7
2,Private,89814,HS-grad,9
3,Local-gov,336951,Assoc-acdm,12
4,Private,160323,Some-college,10
5,missing,103497,Some-college,10
6,Private,198693,10th,6
7,missing,227026,HS-grad,9
8,Self-emp-not-inc,104626,Prof-school,15
9,Private,369667,Some-college,10


In [51]:
# select columns by regex
select(df,r"^c")

Row,capital_gain,capital_loss,class
,Int64,Int64,String7
1,0,0,<=50K
2,0,0,<=50K
3,0,0,>50K
4,7688,0,>50K
5,0,0,<=50K
6,0,0,<=50K
7,0,0,<=50K
8,3103,0,>50K
9,0,0,<=50K


In [56]:
# select averything but the age column
select(df, Not(:age))

Row,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,Hungary,⋯
,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,139

In [57]:
# select all the columns between occupation and sex
select(df, Between(:occupation, :sex))

Row,occupation,relationship,race,sex
,String31?,String15,String31,String7
1,Machine-op-inspct,Own-child,Black,Male
2,Farming-fishing,Husband,White,Male
3,Protective-serv,Husband,White,Male
4,Machine-op-inspct,Husband,Black,Male
5,missing,Own-child,White,Female
6,Other-service,Not-in-family,White,Male
7,missing,Unmarried,Black,Male
8,Prof-specialty,Husband,White,Male
9,Other-service,Unmarried,White,Female


In [59]:
# select a copy of the columns sex and age
df[:,[:sex,:age]]

Row,sex,age
,String7,Int64
1,Male,25
2,Male,38
3,Male,28
4,Male,44
5,Female,18
6,Male,34
7,Male,29
8,Male,63
9,Female,24


In [60]:
# Select original column vectors
df[!,[:sex, :age]]

Row,sex,age
,String7,Int64
1,Male,25
2,Male,38
3,Male,28
4,Male,44
5,Female,18
6,Male,34
7,Male,29
8,Male,63
9,Female,24


In [61]:
# names(df)\
names(df)

110-element Vector{String}:
 "age"
 "workclass"
 "fnlwgt"
 "education"
 "education_num"
 "marital_status"
 "occupation"
 "relationship"
 "race"
 "sex"
 "capital_gain"
 "capital_loss"
 "hours_per_week"
 ⋮
 "Ecuador"
 "Yugoslavia"
 "Hungary"
 "Hong"
 "Greece"
 "Trinadad&Tobago"
 "Outlying_US(Guam_USVI_etc)"
 "France"
 "Holand_Netherlands"
 "<=50K"
 ">50K"
 "Total"

In [62]:
propertynames(df)

110-element Vector{Symbol}:
 :age
 :workclass
 :fnlwgt
 :education
 :education_num
 :marital_status
 :occupation
 :relationship
 :race
 :sex
 :capital_gain
 :capital_loss
 :hours_per_week
 ⋮
 :Ecuador
 :Yugoslavia
 :Hungary
 :Hong
 :Greece
 Symbol("Trinadad&Tobago")
 Symbol("Outlying_US(Guam_USVI_etc)")
 :France
 :Holand_Netherlands
 Symbol("<=50K")
 Symbol(">50K")
 :Total

In [63]:
# Number of rows
nrow(df)

48842

In [64]:
# Number of columns
ncol(df)

110

In [67]:
# Column index
columnindex(df,:sex)
columnindex(df,"sex")

10

In [68]:
dropmissing(df)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31,Int64,String15,Int64,String31,String31,String15,String31,String7,Int64,Int64,Int64,String31,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K,4912,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,4912,4912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0

In [72]:
# Drop colmns with missing data
dropmissing(df,[:relationship,:occupation])

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,1390,0,1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K,4912,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,0,0,0,0,0,0,0,4912,0,0,0,0,4912,4912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K,1395,0,0,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0,0,0,1395,0,0,0,0,0,0,0,0

In [75]:
allowmissing(df)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64?,String31?,Int64?,String15?,Int64?,String31?,String31?,String15?,String31?,String7?,Int64?,Int64?,Int64?,String31?,String7?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,18,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,

In [76]:
allowmissing(df,:occupation)

Row,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,class,Private,Local_gov,Self_emp_not_inc,Federal_gov,State_gov,Self_emp_inc,Without_pay,11th,HS_grad,Assoc_acdm,Some_college,10th,Prof_school,7th_8th,Bachelors,Masters,5th_6th,Assoc_voc,9th,Doctorate,12th,1st_4th,Preschool,Never_married,Married_civ_spouse,Widowed,Separated,Divorced,Married_spouse_absent,Married_AF_spouse,Machine_op_inspct,Farming_fishing,Protective_serv,Other_service,Prof_specialty,Craft_repair,Adm_clerical,Exec_managerial,Tech_support,Sales,Priv_house_serv,Transport_moving,Handlers_cleaners,Armed_Forces,Black,White,Other,Amer_Indian_Eskimo,Asian_Pac_Islander,Male,Female,United_States,Peru,Guatemala,Mexico,Dominican_Republic,Ireland,Germany,Philippines,Thailand,Haiti,El_Salvador,Puerto_Rico,Vietnam,South,Columbia,Japan,India,Cambodia,Poland,Laos,England,Cuba,Taiwan,Italy,Canada,Portugal,China,Nicaragua,Honduras,Iran,Scotland,Jamaica,Ecuador,Yugoslavia,⋯
,Int64,String31?,Int64,String15,Int64,String31,String31?,String15,String31,String7,Int64,Int64,Int64,String31?,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,0,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,3014,0,0,0,0,3014,0,3014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
2,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1194,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,0,0,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,1194,0,0,0,1194,0,1194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
3,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,4478,0,0,0,4478,0,4478,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
4,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,2130,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,2130,0,0,0,0,2130,0,2130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
5,18,missing,103497,Some-college,10,Never-married,missing,Own-child,White,Female,0,0,30,United-States,<=50K,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1375,0,0,0,0,1375,1375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
6,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K,2640,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,2640,0,0,0,0,0,0,0,0,0,0,0,2640,0,0,0,2640,0,2640,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
7,29,missing,227026,HS-grad,9,Never-married,missing,Unmarried,Black,Male,0,0,40,United-States,<=50K,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3017,0,0,0,0,3017,0,3017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,⋯
8,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0,1390,0,0,0,0,0,0,0,0,0

In [78]:
disallowmissing(df,:occupation)

LoadError: ArgumentError: Missing value found in column :occupation in row 5

In [81]:
disallowmissing(df, [:age,:occupation])

LoadError: ArgumentError: Missing value found in column :occupation in row 5

In [82]:
completecases(df)

48842-element BitVector:
 1
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [84]:
completecases(df,[:age,:occupation])

48842-element BitVector:
 1
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [11]:
select(df,:Male => cumsum)

Row,Male_cumsum
,Int64
1,3014
2,4208
3,8686
4,10816
5,10816
6,13456
7,16473
8,17863
9,17863


In [12]:
select(df,:Male => cumprod)

Row,Male_cumprod
,Int64
1,3014
2,3598716
3,16115050248
4,34325057028240
5,0
6,0
7,0
8,0
9,0


In [20]:
select(dropmissing(df), :occupation => v -> accumulate(min,v))

Row,occupation_function
,String31
1,Machine-op-inspct
2,Farming-fishing
3,Farming-fishing
4,Farming-fishing
5,Farming-fishing
6,Farming-fishing
7,Farming-fishing
8,Craft-repair
9,Craft-repair


In [19]:
select(dropmissing(df), :occupation => v -> accumulate(max,v))

Row,occupation_function
,String31
1,Machine-op-inspct
2,Machine-op-inspct
3,Protective-serv
4,Protective-serv
5,Protective-serv
6,Protective-serv
7,Protective-serv
8,Protective-serv
9,Protective-serv
